# WARNING

# PLAY BY PLAY DATA NOT COMPLETE OR INACCURATE

In [2]:
import os
import requests
import pandas as pd

# BOX SCORE AND PLAY-BY-PLAY

#### USING

**https://github.com/nflverse/nflfastR**



#### OLDER VERSION

**https://github.com/nflverse/nflfastR**

**https://www.nflfastr.com/articles/nflfastR.html**

**https://github.com/nflverse/nflverse-data/releases/tag/pbp**



In [2]:
### Scrape play-by-play data 

!Rscript play-by-play-scrape.R

# Create box scores from play-by-play

In [3]:
### Create box scores from play-by-play data (single season 2022)

# Load the CSV file into a DataFrame
file_path = './data-raw-git-2/play_by_play_2022.csv'  # Update the directory path
df = pd.read_csv(file_path, low_memory=False)

# Group by game_id, quarter ('qtr'), and the team that scored, then count the number of each type of scoring play
grouped_scores = df.groupby(['game_id', 'qtr', 'posteam'])[['touchdown', 'field_goal_result', 'safety', 'extra_point_result']].agg({
    'touchdown': 'sum',
    'field_goal_result': lambda x: (x == 'made').sum(),
    'safety': 'sum',
    'extra_point_result': lambda x: (x == 'good').sum()
}).reset_index()

# Calculate the score for each group
grouped_scores['score'] = grouped_scores['touchdown']*6 + grouped_scores['field_goal_result']*3 + grouped_scores['safety']*2 + grouped_scores['extra_point_result']

# Pivot the DataFrame to create the box score format, filling in missing values with 0
box_scores = grouped_scores.pivot_table(index=['game_id'], columns=['posteam', 'qtr'], values='score', aggfunc='sum').fillna(0)

# Reset the column names for easier access
box_scores.columns = [f"{col[0]}_Q{col[1]}" for col in box_scores.columns]

# Add a total score column for each team
unique_teams = grouped_scores['posteam'].unique()
for team in unique_teams:
    cols = [col for col in box_scores.columns if col.startswith(team)]
    box_scores[f"{team}_Total"] = box_scores[cols].sum(axis=1)

# Merge the box_scores DataFrame with the original DataFrame to get home and away team information for each game_id
game_info = df[['game_id', 'home_team', 'away_team']].drop_duplicates()
box_scores_with_teams = box_scores.reset_index().merge(game_info, on='game_id', how='left')

# Initialize new columns for home and away scores by quarter and total
quarters = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5']  # Including Q5 for overtime
new_columns = [f'home_{qtr}' for qtr in quarters] + [f'away_{qtr}' for qtr in quarters] + ['home_Total', 'away_Total']

# Populate the new columns
for index, row in box_scores_with_teams.iterrows():
    home_team = row['home_team']
    away_team = row['away_team']
    
    for qtr in quarters:
        home_col = f"{home_team}_{qtr}"
        away_col = f"{away_team}_{qtr}"
        
        box_scores_with_teams.loc[index, f'home_{qtr}'] = row.get(home_col, 0)
        box_scores_with_teams.loc[index, f'away_{qtr}'] = row.get(away_col, 0)
    
    home_total_col = f"{home_team}_Total"
    away_total_col = f"{away_team}_Total"
    
    box_scores_with_teams.loc[index, 'home_Total'] = row.get(home_total_col, 0)
    box_scores_with_teams.loc[index, 'away_Total'] = row.get(away_total_col, 0)

# Keep only relevant columns
final_box_scores = box_scores_with_teams[['game_id', 'home_team', 'away_team'] + new_columns]

# Save to CSV
output_file_path = './data-raw-git-2/box_scores_2022.csv'  # Update the directory path
final_box_scores.to_csv(output_file_path, index=False)


In [3]:
### Create box scores from play-by-play data (single season 2021)

# Load the CSV file into a DataFrame
file_path = './data-raw-git-2/play_by_play_2021.csv'  # Update the directory path
df = pd.read_csv(file_path, low_memory=False)

# Group by game_id, quarter ('qtr'), and the team that scored, then count the number of each type of scoring play
grouped_scores = df.groupby(['game_id', 'qtr', 'posteam'])[['touchdown', 'field_goal_result', 'safety', 'extra_point_result']].agg({
    'touchdown': 'sum',
    'field_goal_result': lambda x: (x == 'made').sum(),
    'safety': 'sum',
    'extra_point_result': lambda x: (x == 'good').sum()
}).reset_index()

# Calculate the score for each group
grouped_scores['score'] = grouped_scores['touchdown']*6 + grouped_scores['field_goal_result']*3 + grouped_scores['safety']*2 + grouped_scores['extra_point_result']

# Pivot the DataFrame to create the box score format, filling in missing values with 0
box_scores = grouped_scores.pivot_table(index=['game_id'], columns=['posteam', 'qtr'], values='score', aggfunc='sum').fillna(0)

# Reset the column names for easier access
box_scores.columns = [f"{col[0]}_Q{col[1]}" for col in box_scores.columns]

# Add a total score column for each team
unique_teams = grouped_scores['posteam'].unique()
for team in unique_teams:
    cols = [col for col in box_scores.columns if col.startswith(team)]
    box_scores[f"{team}_Total"] = box_scores[cols].sum(axis=1)

# Merge the box_scores DataFrame with the original DataFrame to get home and away team information for each game_id
game_info = df[['game_id', 'home_team', 'away_team']].drop_duplicates()
box_scores_with_teams = box_scores.reset_index().merge(game_info, on='game_id', how='left')

# Initialize new columns for home and away scores by quarter and total
quarters = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5']  # Including Q5 for overtime
new_columns = [f'home_{qtr}' for qtr in quarters] + [f'away_{qtr}' for qtr in quarters] + ['home_Total', 'away_Total']

# Populate the new columns
for index, row in box_scores_with_teams.iterrows():
    home_team = row['home_team']
    away_team = row['away_team']
    
    for qtr in quarters:
        home_col = f"{home_team}_{qtr}"
        away_col = f"{away_team}_{qtr}"
        
        box_scores_with_teams.loc[index, f'home_{qtr}'] = row.get(home_col, 0)
        box_scores_with_teams.loc[index, f'away_{qtr}'] = row.get(away_col, 0)
    
    home_total_col = f"{home_team}_Total"
    away_total_col = f"{away_team}_Total"
    
    box_scores_with_teams.loc[index, 'home_Total'] = row.get(home_total_col, 0)
    box_scores_with_teams.loc[index, 'away_Total'] = row.get(away_total_col, 0)

# Keep only relevant columns
final_box_scores = box_scores_with_teams[['game_id', 'home_team', 'away_team'] + new_columns]

# Save to CSV
output_file_path = './data-raw-git-2/box_scores_2021.csv'  # Update the directory path
final_box_scores.to_csv(output_file_path, index=False)


In [4]:
### Create box scores from play-by-play data (single season 2020)

# Load the CSV file into a DataFrame
file_path = './data-raw-git-2/play_by_play_2020.csv'  # Update the directory path
df = pd.read_csv(file_path, low_memory=False)

# Group by game_id, quarter ('qtr'), and the team that scored, then count the number of each type of scoring play
grouped_scores = df.groupby(['game_id', 'qtr', 'posteam'])[['touchdown', 'field_goal_result', 'safety', 'extra_point_result']].agg({
    'touchdown': 'sum',
    'field_goal_result': lambda x: (x == 'made').sum(),
    'safety': 'sum',
    'extra_point_result': lambda x: (x == 'good').sum()
}).reset_index()

# Calculate the score for each group
grouped_scores['score'] = grouped_scores['touchdown']*6 + grouped_scores['field_goal_result']*3 + grouped_scores['safety']*2 + grouped_scores['extra_point_result']

# Pivot the DataFrame to create the box score format, filling in missing values with 0
box_scores = grouped_scores.pivot_table(index=['game_id'], columns=['posteam', 'qtr'], values='score', aggfunc='sum').fillna(0)

# Reset the column names for easier access
box_scores.columns = [f"{col[0]}_Q{col[1]}" for col in box_scores.columns]

# Add a total score column for each team
unique_teams = grouped_scores['posteam'].unique()
for team in unique_teams:
    cols = [col for col in box_scores.columns if col.startswith(team)]
    box_scores[f"{team}_Total"] = box_scores[cols].sum(axis=1)

# Merge the box_scores DataFrame with the original DataFrame to get home and away team information for each game_id
game_info = df[['game_id', 'home_team', 'away_team']].drop_duplicates()
box_scores_with_teams = box_scores.reset_index().merge(game_info, on='game_id', how='left')

# Initialize new columns for home and away scores by quarter and total
quarters = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5']  # Including Q5 for overtime
new_columns = [f'home_{qtr}' for qtr in quarters] + [f'away_{qtr}' for qtr in quarters] + ['home_Total', 'away_Total']

# Populate the new columns
for index, row in box_scores_with_teams.iterrows():
    home_team = row['home_team']
    away_team = row['away_team']
    
    for qtr in quarters:
        home_col = f"{home_team}_{qtr}"
        away_col = f"{away_team}_{qtr}"
        
        box_scores_with_teams.loc[index, f'home_{qtr}'] = row.get(home_col, 0)
        box_scores_with_teams.loc[index, f'away_{qtr}'] = row.get(away_col, 0)
    
    home_total_col = f"{home_team}_Total"
    away_total_col = f"{away_team}_Total"
    
    box_scores_with_teams.loc[index, 'home_Total'] = row.get(home_total_col, 0)
    box_scores_with_teams.loc[index, 'away_Total'] = row.get(away_total_col, 0)

# Keep only relevant columns
final_box_scores = box_scores_with_teams[['game_id', 'home_team', 'away_team'] + new_columns]

# Save to CSV
output_file_path = './data-raw-git-2/box_scores_2020.csv'  # Update the directory path
final_box_scores.to_csv(output_file_path, index=False)


In [ ]:
### Looping through all years play-by-play data

# Define the range of years you want to process
start_year = 1999
end_year = 2023

# List of years to process within the specified range
years = list(range(start_year, end_year + 1))

for year in years:
    # Construct the file path for the CSV file of the current year
    file_path = f'./data-raw-git-2/play_by_play_{year}.csv'
    
    # Load the CSV file into a DataFrame
    df = pd.read_csv(file_path, low_memory=False)
    
    # Group by game_id, quarter ('qtr'), and the team that scored, then count the number of each type of scoring play
    grouped_scores = df.groupby(['game_id', 'qtr', 'posteam'])[['touchdown', 'field_goal_result', 'safety', 'extra_point_result']].agg({
        'touchdown': 'sum',
        'field_goal_result': lambda x: (x == 'made').sum(),
        'safety': 'sum',
        'extra_point_result': lambda x: (x == 'good').sum()
    }).reset_index()

    # Calculate the score for each group
    grouped_scores['score'] = grouped_scores['touchdown']*6 + grouped_scores['field_goal_result']*3 + grouped_scores['safety']*2 + grouped_scores['extra_point_result']

    # Pivot the DataFrame to create the box score format, filling in missing values with 0
    box_scores = grouped_scores.pivot_table(index=['game_id'], columns=['posteam', 'qtr'], values='score', aggfunc='sum').fillna(0)

    # Reset the column names for easier access
    box_scores.columns = [f"{col[0]}_Q{col[1]}" for col in box_scores.columns]

    # Add a total score column for each team
    unique_teams = grouped_scores['posteam'].unique()
    for team in unique_teams:
        cols = [col for col in box_scores.columns if col.startswith(team)]
        box_scores[f"{team}_Total"] = box_scores[cols].sum(axis=1)

    # Merge the box_scores DataFrame with the original DataFrame to get home and away team information for each game_id
    game_info = df[['game_id', 'home_team', 'away_team']].drop_duplicates()
    box_scores_with_teams = box_scores.reset_index().merge(game_info, on='game_id', how='left')

    # Initialize new columns for home and away scores by quarter and total
    quarters = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5']  # Including Q5 for overtime
    new_columns = [f'home_{qtr}' for qtr in quarters] + [f'away_{qtr}' for qtr in quarters] + ['home_Total', 'away_Total']

    # Populate the new columns
    for index, row in box_scores_with_teams.iterrows():
        home_team = row['home_team']
        away_team = row['away_team']

        for qtr in quarters:
            home_col = f"{home_team}_{qtr}"
            away_col = f"{away_team}_{qtr}"

            box_scores_with_teams.loc[index, f'home_{qtr}'] = row.get(home_col, 0)
            box_scores_with_teams.loc[index, f'away_{qtr}'] = row.get(away_col, 0)

        home_total_col = f"{home_team}_Total"
        away_total_col = f"{away_team}_Total"

        box_scores_with_teams.loc[index, 'home_Total'] = row.get(home_total_col, 0)
        box_scores_with_teams.loc[index, 'away_Total'] = row.get(away_total_col, 0)

    # Keep only relevant columns
    final_box_scores = box_scores_with_teams[['game_id', 'home_team', 'away_team'] + new_columns]
    
    # Print the current file being processed
    print(f"Processing file: ./data-raw-git-2/box_scores_{year}.csv")
    
    # Save to CSV
    final_box_scores.to_csv(f'./data-raw-git-2/box_scores_{year}.csv', index=False)


In [ ]:
### ADD TD Scorers to box scores

def append_touchdown_scorers(box_scores_csv, play_by_play_csv, output_csv):
    # Load the original box score and play-by-play DataFrames
    df_box_scores = pd.read_csv(box_scores_csv)
    df_play_by_play = pd.read_csv(play_by_play_csv, low_memory=False)

    # Filter rows where a touchdown has been scored
    df_touchdowns = df_play_by_play[df_play_by_play['touchdown'] == 1][['game_id', 'td_player_name', 'td_player_id']]
    
    # Group by 'game_id' and aggregate touchdown information into lists
    grouped_touchdowns = df_touchdowns.groupby('game_id').agg({
        'td_player_name': list,
        'td_player_id': list
    }).reset_index()

    # Merge the original box score DataFrame with the grouped_touchdowns DataFrame based on 'game_id'
    df_box_scores_merged = pd.merge(df_box_scores, grouped_touchdowns, on='game_id', how='left')
    
    # Populate new columns for touchdown scorers and their IDs
    for index, row in df_box_scores_merged.iterrows():
        td_scorers = row.get('td_player_name', [])
        td_scorer_ids = row.get('td_player_id', [])
        
        if not isinstance(td_scorers, list):
            td_scorers = []
        if not isinstance(td_scorer_ids, list):
            td_scorer_ids = []

        for i, (scorer, scorer_id) in enumerate(zip(td_scorers, td_scorer_ids)):
            df_box_scores_merged.loc[index, f'TD{i+1}-scorer'] = scorer
            df_box_scores_merged.loc[index, f'TD{i+1}-scorer-id'] = scorer_id
    
    # Drop the original 'td_player_name' and 'td_player_id' columns
    df_box_scores_merged.drop(columns=['td_player_name', 'td_player_id'], inplace=True, errors='ignore')
    
    # Save the updated DataFrame to a new CSV file
    df_box_scores_merged.to_csv(output_csv, index=False)


In [ ]:
# Example usage
append_touchdown_scorers('./data-raw-git-2/box_scores_2022.csv', './data-raw-git-2/play_by_play_2022.csv', './data-raw-git-2/box_scores_with_touchdowns_2022.csv')

In [ ]:
### Without function

# Define the paths to your original box score CSV and play-by-play CSV within the directory
box_scores_csv = './data-raw-git-2/box_scores_2022.csv'
play_by_play_csv = './data-raw-git-2/play_by_play_2022.csv'
output_csv = './data-raw-git-2/box_scores_with_touchdowns_2022.csv'

# Load the original box score and play-by-play DataFrames
df_box_scores = pd.read_csv(box_scores_csv)
df_play_by_play = pd.read_csv(play_by_play_csv, low_memory=False)

# Filter rows where a touchdown has been scored
df_touchdowns = df_play_by_play[df_play_by_play['touchdown'] == 1][['game_id', 'td_player_name', 'td_player_id']]

# Group by 'game_id' and aggregate touchdown information into lists
grouped_touchdowns = df_touchdowns.groupby('game_id').agg({
    'td_player_name': list,
    'td_player_id': list
}).reset_index()

# Merge the original box score DataFrame with the grouped_touchdowns DataFrame based on 'game_id'
df_box_scores_merged = pd.merge(df_box_scores, grouped_touchdowns, on='game_id', how='left')

# Populate new columns for touchdown scorers and their IDs
for index, row in df_box_scores_merged.iterrows():
    td_scorers = row.get('td_player_name', [])
    td_scorer_ids = row.get('td_player_id', [])
    
    if not isinstance(td_scorers, list):
        td_scorers = []
    if not isinstance(td_scorer_ids, list):
        td_scorer_ids = []
        
    for i, (scorer, scorer_id) in enumerate(zip(td_scorers, td_scorer_ids)):
        df_box_scores_merged.loc[index, f'TD{i+1}-scorer'] = scorer
        df_box_scores_merged.loc[index, f'TD{i+1}-scorer-id'] = scorer_id

# Drop the original 'td_player_name' and 'td_player_id' columns
df_box_scores_merged.drop(columns=['td_player_name', 'td_player_id'], inplace=True, errors='ignore')

# Save the updated DataFrame to a new CSV file
df_box_scores_merged.to_csv(output_csv, index=False)


In [ ]:
## ADD TD Scorers Testing 1

# Load the CSV file into a DataFrame
file_path = './data-raw-git-2/play_by_play_2022.csv'  # Update the directory path
df = pd.read_csv(file_path, low_memory=False)

# Load the CSV file into a DataFrame (for TD scorers) 
#file_path = '/mnt/data/play_by_play_2022.csv'
df_2022 = pd.read_csv('./data-raw-git-2/play_by_play_2022.csv', low_memory=False)

# Filter the DataFrame to only contain plays where a touchdown occurred
td_df = df_2022[df_2022['touchdown'] == 1]

# Step 2: Order by Game and Quarter
# Sort these touchdown records by game_id and qtr
td_df = td_df.sort_values(by=['game_id', 'qtr'])

# Step 3: Generate TD Scorer Columns
# Initialize an empty dictionary to hold the new columns
td_scorers_dict = {}

# Loop through each game and quarter to generate the new columns
for game_id, game_data in td_df.groupby('game_id'):
    for qtr, qtr_data in game_data.groupby('qtr'):
        key = (game_id, qtr)
        td_scorers_dict[key] = []
        
        # Loop through the DataFrame rows to get the td_player_name and td_player_id
        for i, row in enumerate(qtr_data.itertuples()):
            td_scorers_dict[key].append({
                f'TD{i+1}-scorer': row.td_player_name,
                f'TD{i+1}-scorer-id': row.td_player_id
            })

# Convert the dictionary to a DataFrame for easier merging
td_scorers_df = pd.DataFrame([
    {'game_id': game_id, 'qtr': qtr, **scorer}
    for (game_id, qtr), scorers in td_scorers_dict.items()
    for scorer in scorers
])


# Group by game_id, quarter ('qtr'), and the team that scored, then count the number of each type of scoring play
grouped_scores = df.groupby(['game_id', 'qtr', 'posteam'])[['touchdown', 'field_goal_result', 'safety', 'extra_point_result']].agg({
    'touchdown': 'sum',
    'field_goal_result': lambda x: (x == 'made').sum(),
    'safety': 'sum',
    'extra_point_result': lambda x: (x == 'good').sum()
}).reset_index()

# Calculate the score for each group
grouped_scores['score'] = grouped_scores['touchdown']*6 + grouped_scores['field_goal_result']*3 + grouped_scores['safety']*2 + grouped_scores['extra_point_result']

# Pivot the DataFrame to create the box score format, filling in missing values with 0
box_scores = grouped_scores.pivot_table(index=['game_id'], columns=['posteam', 'qtr'], values='score', aggfunc='sum').fillna(0)

# Reset the column names for easier access
box_scores.columns = [f"{col[0]}_Q{col[1]}" for col in box_scores.columns]

# Add a total score column for each team
unique_teams = grouped_scores['posteam'].unique()
for team in unique_teams:
    cols = [col for col in box_scores.columns if col.startswith(team)]
    box_scores[f"{team}_Total"] = box_scores[cols].sum(axis=1)

# Merge the box_scores DataFrame with the original DataFrame to get home and away team information for each game_id
game_info = df[['game_id', 'home_team', 'away_team']].drop_duplicates()
box_scores_with_teams = box_scores.reset_index().merge(game_info, on='game_id', how='left')

# Initialize new columns for home and away scores by quarter and total
quarters = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5']  # Including Q5 for overtime
new_columns = [f'home_{qtr}' for qtr in quarters] + [f'away_{qtr}' for qtr in quarters] + ['home_Total', 'away_Total']

# Populate the new columns
for index, row in box_scores_with_teams.iterrows():
    home_team = row['home_team']
    away_team = row['away_team']
    
    for qtr in quarters:
        home_col = f"{home_team}_{qtr}"
        away_col = f"{away_team}_{qtr}"
        
        box_scores_with_teams.loc[index, f'home_{qtr}'] = row.get(home_col, 0)
        box_scores_with_teams.loc[index, f'away_{qtr}'] = row.get(away_col, 0)
    
    home_total_col = f"{home_team}_Total"
    away_total_col = f"{away_team}_Total"
    
    box_scores_with_teams.loc[index, 'home_Total'] = row.get(home_total_col, 0)
    box_scores_with_teams.loc[index, 'away_Total'] = row.get(away_total_col, 0)

# Keep only relevant columns
final_box_scores = box_scores_with_teams[['game_id'] + new_columns]
#final_box_scores = box_scores_with_teams[['game_id', 'home_team', 'away_team'] + new_columns]
#######
# Merge the box_scores DataFrame with td_scorers_df to include the touchdown scorers information
final_box_scores_with_td = pd.merge(final_box_scores, td_scorers_df, on=['game_id'], how='left')
#######

# Save to CSV
# output_file_path = './data-raw-git-2/box_scores_2022.csv'  # Update the directory path
# final_box_scores.to_csv(output_file_path, index=False)
output_file_path = './data-raw-git-2/box_scores_2022_testing.csv'
final_box_scores_with_td.to_csv(output_file_path, index=False)


In [ ]:
### Add TD scorers testing 1

# Load the 2022 NFL play-by-play data into a DataFrame
file_path = '/mnt/data/play_by_play_2022.csv'
df_2022 = pd.read_csv(file_path, low_memory=False)

# Filter the DataFrame to only contain plays where a touchdown occurred
td_df = df_2022[df_2022['touchdown'] == 1]

# Step 2: Order by Game and Quarter
# Sort these touchdown records by game_id and qtr
td_df = td_df.sort_values(by=['game_id', 'qtr'])

# Step 3: Generate TD Scorer Columns
# Initialize an empty dictionary to hold the new columns
td_scorers_dict = {}

# Loop through each game and quarter to generate the new columns
for game_id, game_data in td_df.groupby('game_id'):
    for qtr, qtr_data in game_data.groupby('qtr'):
        key = (game_id, qtr)
        td_scorers_dict[key] = []
        
        # Loop through the DataFrame rows to get the td_player_name and td_player_id
        for i, row in enumerate(qtr_data.itertuples()):
            td_scorers_dict[key].append({
                f'TD{i+1}-scorer': row.td_player_name,
                f'TD{i+1}-scorer-id': row.td_player_id
            })

# Convert the dictionary to a DataFrame for easier merging
td_scorers_df = pd.DataFrame([
    {'game_id': game_id, 'qtr': qtr, **scorer}
    for (game_id, qtr), scorers in td_scorers_dict.items()
    for scorer in scorers
])



In [ ]:
### Raw GPT first step WORKING 

# Load the CSV file into a DataFrame
file_path = './data-raw-git-2/play_by_play_2022.csv'
df = pd.read_csv(file_path)

# Group by game_id, quarter ('qtr'), and the team that scored, then count the number of each type of scoring play
grouped_scores = scoring_plays.groupby(['game_id', 'qtr', 'posteam'])[['touchdown', 'field_goal_result', 'safety', 'extra_point_result']].agg({
    'touchdown': 'sum',
    'field_goal_result': lambda x: (x == 'made').sum(),
    'safety': 'sum',
    'extra_point_result': lambda x: (x == 'good').sum()
}).reset_index()

# Calculate the score for each group
# Note: We assume 6 points for a touchdown, 3 for a field goal, 2 for a safety, and 1 for an extra point
grouped_scores['score'] = grouped_scores['touchdown']*6 + grouped_scores['field_goal_result']*3 + grouped_scores['safety']*2 + grouped_scores['extra_point_result']

# Pivot the DataFrame to create the box score format, filling in missing values with 0
box_scores = grouped_scores.pivot_table(index=['game_id'], columns=['posteam', 'qtr'], values='score', aggfunc='sum').fillna(0)

# Reset the column names for easier access
box_scores.columns = [f"{col[0]}_Q{col[1]}" for col in box_scores.columns]

# Add a total score column for each team
unique_teams = grouped_scores['posteam'].unique()
for team in unique_teams:
    cols = [col for col in box_scores.columns if col.startswith(team)]
    box_scores[f"{team}_Total"] = box_scores[cols].sum(axis=1)

# Save the final product to a new CSV file
output_file_path = './data-raw-git-2/box_scores_2022_testing.csv'
box_scores.reset_index().to_csv(output_file_path, index=False)

box_scores.head(), output_file_path


In [ ]:
### Raw GPT step 2 WORKING

# Merge the box_scores DataFrame with the original DataFrame to get home and away team information for each game_id
game_info = df[['game_id', 'home_team', 'away_team']].drop_duplicates()
box_scores_with_teams = box_scores.reset_index().merge(game_info, on='game_id', how='left')

# Initialize new columns for home and away scores by quarter and total
quarters = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5']  # Including Q5 for overtime
new_columns = [f'home_{qtr}' for qtr in quarters] + [f'away_{qtr}' for qtr in quarters] + ['home_Total', 'away_Total']

# Populate the new columns
for index, row in box_scores_with_teams.iterrows():
    home_team = row['home_team']
    away_team = row['away_team']
    
    for qtr in quarters:
        home_col = f"{home_team}_{qtr}"
        away_col = f"{away_team}_{qtr}"
        
        box_scores_with_teams.loc[index, f'home_{qtr}'] = row.get(home_col, 0)
        box_scores_with_teams.loc[index, f'away_{qtr}'] = row.get(away_col, 0)
    
    home_total_col = f"{home_team}_Total"
    away_total_col = f"{away_team}_Total"
    
    box_scores_with_teams.loc[index, 'home_Total'] = row.get(home_total_col, 0)
    box_scores_with_teams.loc[index, 'away_Total'] = row.get(away_total_col, 0)

# Keep only relevant columns
final_box_scores = box_scores_with_teams[['game_id', 'home_team', 'away_team'] + new_columns]

# Save the restructured DataFrame to a new CSV file
output_file_path_restructured = './data-raw-git-2/restructured_box_scores_2022.csv'
final_box_scores.to_csv(output_file_path_restructured, index=False)

final_box_scores.head(), output_file_path_restructured
